### Import packages

In [ ]:
import pandas as pd
import numpy as np
import re as re
import nltk
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
import csv
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
from itertools import cycle
from scipy import interp
from sklearn.preprocessing import StandardScaler
# generate classification report using predictions for categorical model
from sklearn.metrics import classification_report, accuracy_score , f1_score, precision_score

### Labeling of the splitted reviews
The splitted reviews have been labelled using the rule based approach. The keywords for each topics are defined and use to lebel the reviews

In [ ]:
#0: delivery, 1: service, 2:price, 3: product quality
#Each topics and their respective keywords
delivery = ['delivery', 'condition', 'package', 'pack', 'fast', 'slow', 'ship', 'bubble', 'wrap', 'envelope', 'slow','parcel','delivered','arrive','courier','bubblewrap','damage','early','earlier','earlier','receive']

service = ['reply', 'message', 'service', 'response', 'chat', 'responsive', 'irresponsible', 'unethical', 'contact', 'enquiry', 'unresponsive','friendly','resolve','respond','query']

price = ['price', 'sale', 'cheap', 'worth', 'deal', 'flash', 'gift', 'freebie', 'promotion', 'expensive', 'affordable','discount','free','cheaper','promo','value','offer','money','voucher','pricey']

quality = ['expiry','manufacture','smell', 'smooth', 'sticky','hydrate','effective','last','lasting','apply','application','colour','color','easy','oily','coverage','sensitive','texture','soft',
           'frizzy','quality','dry','effect','oil','skin', 'love','like','satisfied','feel','felt','pores','tight','mask','authentic','expiring','lighten','shade','bright','mascara','lash','waterproof',
           'lightweight','foundation','suitable','exp','shelf','manufacture','nice','recommend','result','meg','oskinriginal','again','fake','pigment','scent','dryness','stain','smudge','lip','lipstick','pretty',
           'fragrance','promising','hair','repeat','happy','made','moisture','remove','volume','suit','awesome','hope','great','content','clean','work','cute','fit','fantastic','genuine','powder','hard',
           'real','eyeliner','gently','makeup','amazing','improvement','allergic','foam','cleanse','best','rubbing','rub','wonderful','water','stay','wear','transfer','wearing','disappoint','crease','cover','conceal',
          'concealer','reapply','useful','old','picture','test','crack','soft']

no_topic = ['no comments review is an image', 'no review receive', 'your review has been hidden due to inappropriate content note hope may remove the coins awarded for this review']
no_topic_words = ['try','thank','tried', 'describe','hope']
stemmer = PorterStemmer()

#Stemming of the keywords for each topic
delivery_stem = [stemmer.stem(w) for w in delivery]
service_stem = [stemmer.stem(w) for w in service]
price_stem = [stemmer.stem(w) for w in price]
quality_stem = [stemmer.stem(w) for w in quality]
no_topic_stem = [stemmer.stem(w) for w in no_topic_words]


data_file = 'data/all_data_clean_split.csv'
df = pd.read_csv(data_file)
df

review_list =df.Review_splitted.tolist()

review_label = []
topic_num = []
topic_name = []

#Labelling of the splitted reviews based on the keywords
for i in range(len(review_list)):
    review_raw = review_list[i]
    if review_raw in no_topic:
        review_label.append(review_raw)
        topic_num.append(4)
        topic_name.append('no_topic')
    else:
        review = nltk.word_tokenize(review_list[i])
        review_stemmed = [stemmer.stem(w) for w in review]
        delivery_count = 0
        service_count = 0
        price_count = 0
        quality_count = 0
        no_topic_count = 0
        for word in review_stemmed:
            if word in delivery_stem:
                delivery_count += 1
            elif word in service_stem:
                service_count += 1
            elif word in price_stem:
                price_count += 1
            elif word in quality_stem:
                quality_count += 1
            elif word in no_topic_stem:
                no_topic_count += 1
                
        #Label is only given to straighforward reviews that only contains 1 topic. This is to ensure that the classifier
        # is able to learn the classification more accurately
        if delivery_count > 0 and service_count == 0 and price_count == 0 and quality_count == 0 and no_topic_count == 0:

            review_label.append(review_list[i])
            topic_num.append(0)
            topic_name.append('delivery')
        elif delivery_count == 0 and service_count > 0 and price_count == 0 and quality_count == 0 and no_topic_count == 0:
            review_label.append(review_list[i])
            topic_num.append(1)
            topic_name.append('service')
        elif delivery_count == 0 and service_count == 0 and price_count > 0 and quality_count == 0 and no_topic_count == 0:
            review_label.append(review_list[i])
            topic_num.append(2)
            topic_name.append('price')
        elif delivery_count == 0 and service_count == 0 and price_count == 0 and quality_count > 0 and no_topic_count == 0:
            review_label.append(review_list[i])
            topic_num.append(3)
            topic_name.append('quality')
        elif delivery_count == 0 and service_count == 0 and price_count == 0 and quality_count == 0 and no_topic_count > 0:
            review_label.append(review_list[i])
            topic_num.append(4)
            topic_name.append('no_topic')
            
df_with_label = pd.DataFrame({'review': review_label, 'topic_num': topic_num, 'topic_name':topic_name})
# df_with_label.head()
# with_label.to_csv('Data/with_label_v2.csv')
            
            

## Training of classification model for the topics

### Import Packages and Libraries

In [1]:
# from __future__ import absolute_import, division, print_function, unicode_literals
# import tensorflow as tf
# from tensorflow.keras import layers
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import re
import gensim
from gensim import corpora
from gensim import similarities
from gensim import models
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import warnings
warnings.filterwarnings("ignore")

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Data preprocessing

In [35]:
reviews = df_with_label['review']

review_docs = []
for each_reviews in reviews:
    temp = each_reviews.split(" ")
    review_docs.append(temp)
# print (review_docs)

# Make sure all words are in lowercase
reviews_lower = [[each_word.lower() for each_word in each_review] for each_review in review_docs]
# print (reviews_lower)

# Use regular expressions to keep only allphabetical words
reviews_alpha = [[each_word for each_word in each_review if re.search('^[a-z]+$', each_word)] for each_review in reviews_lower]
# print (reviews_alpha)

# Remove stop words
stop_list = stopwords.words('english')
reviews_stop = [[each_word for each_word in each_review if each_word not in stop_list] for each_review in reviews_alpha]
# print (reviews_stop)

# Porter Stemming
stemmer = PorterStemmer()
reviews_stem = [[stemmer.stem(each_word) for each_word in each_review] for each_review in reviews_stop]
# print (reviews_stem)

all_data_cleaned = []
for each_sentence in reviews_stem:
    sentence = ""
    for each_word in each_sentence:
        sentence += each_word + " "
    sentence = sentence[0:-1]
    all_data_cleaned.append(sentence)
# print (all_data_cleaned)

topic_num = data['topic_num']


0        3
1        3
2        0
3        0
4        3
5        3
6        4
7        3
8        0
9        2
10       3
11       0
12       0
13       2
14       3
15       2
16       4
17       3
18       0
19       0
20       0
21       3
22       3
23       0
24       4
25       3
26       2
27       2
28       4
29       0
        ..
23819    3
23820    0
23821    4
23822    4
23823    4
23824    4
23825    4
23826    3
23827    3
23828    1
23829    0
23830    4
23831    4
23832    4
23833    4
23834    4
23835    4
23836    4
23837    3
23838    3
23839    0
23840    3
23841    4
23842    4
23843    4
23844    4
23845    4
23846    4
23847    4
23848    4
Name: topic_num, Length: 23849, dtype: int64


### Building a Model - Multinomial Naive Bayes 
1. Count Vectorizer
2. TFIDF Vectorizer

In [3]:
print ("1. Count Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = False, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

mnbClf = MultinomialNB()
mnbClf.fit(X_train, y_train)
mnbClf_ypred = mnbClf.predict(X_test)
f1_mnbClf = f1_score(y_test, mnbClf_ypred, average = 'weighted')
accuracy_mnbClf = accuracy_score(y_test, mnbClf_ypred)
print ("F1-score of Multinomial Naive Bayes: ", f1_mnbClf*100)
print ("Accuracy of Multinomial Naive Bayes: ", accuracy_mnbClf*100)

print ("\n2. TFIDF Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = True, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

mnbTfidfClf = MultinomialNB()
mnbTfidfClf.fit(X_train, y_train)
mnbTfidfClf_ypred = mnbTfidfClf.predict(X_test)
f1_mnbTfidfClf = f1_score(y_test, mnbTfidfClf_ypred, average='weighted')
accuracy_mnbTfidfClf = accuracy_score(y_test, mnbTfidfClf_ypred)
print ("F1-score of Multinomial Naive Bayes with TFIDF: ", f1_mnbTfidfClf*100)
print ("Accuracy of Multinomial Naive Bayes with TFIDF: ", accuracy_mnbTfidfClf*100)




1. Count Vectorizer
F1-score of Multinomial Naive Bayes:  92.55332206254671
Accuracy of Multinomial Naive Bayes:  92.67147409022304

2. TFIDF Vectorizer
F1-score of Multinomial Naive Bayes with TFIDF:  92.8543552552533
Accuracy of Multinomial Naive Bayes with TFIDF:  92.93979540499748


### Building a Model - Bernoulli Naive Bayes
1. Count Vectorizer
2. TFIDF Vectorizer

In [4]:
print ("1. Count Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = False, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

bnbClf = BernoulliNB()
bnbClf.fit(X_train, y_train)
bnbClf_ypred = bnbClf.predict(X_test)
f1_bnbClf = f1_score(y_test, bnbClf_ypred, average = 'weighted')
accuracy_bnbClf = accuracy_score(y_test, bnbClf_ypred)
print ("F1-score of Bernoulli Naive Bayes: ", f1_bnbClf*100)
print ("Accuracy of Bernoulli Naive Bayes: ", accuracy_bnbClf*100)


print ("\n2. TFIDF Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = True, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

bnbTfidfClf = BernoulliNB()
bnbTfidfClf.fit(X_train, y_train)
bnbTfidfClf_ypred = bnbTfidfClf.predict(X_test)
f1_bnbTfidfClf = f1_score(y_test, bnbTfidfClf_ypred, average='weighted')
accuracy_bnbTfidfClf = accuracy_score(y_test, bnbTfidfClf_ypred)
print ("F1-score of Bernoulli Naive Bayes with TFIDF: ", f1_bnbTfidfClf*100)
print ("Accuracy of Bernoulli Naive Bayes with TFIDF: ", accuracy_bnbTfidfClf*100)

1. Count Vectorizer
F1-score of Bernoulli Naive Bayes:  90.91530178040284
Accuracy of Bernoulli Naive Bayes:  91.0950863659232

2. TFIDF Vectorizer
F1-score of Bernoulli Naive Bayes with TFIDF:  90.91530178040284
Accuracy of Bernoulli Naive Bayes with TFIDF:  91.0950863659232


### Build a Model - Logistic Regression
1. Count Vectorizer
2. TFIDF Vectorizer

In [5]:
print ("1. Count Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = False, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

logRegClf = LogisticRegression()
logRegClf.fit(X_train, y_train)
logRegClf_ypred = logRegClf.predict(X_test)
f1_logRegClf = f1_score(y_test, logRegClf_ypred, average = 'weighted')
accuracy_logRegClf = accuracy_score(y_test, logRegClf_ypred)
print ("F1-score of Logistic Regression: ", f1_logRegClf*100)
print ("Accuracy of Logistic Regression: ", accuracy_logRegClf*100)


print ("\n2. TFIDF Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = True, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

logRegTfidfClf = LogisticRegression()
logRegTfidfClf.fit(X_train, y_train)
logRegTfidfClf_ypred = logRegTfidfClf.predict(X_test)
f1_logRegTfidfClf = f1_score(y_test, logRegTfidfClf_ypred, average='weighted')
accuracy_logRegTfidfClf = accuracy_score(y_test, logRegTfidfClf_ypred)
print ("F1-score of Logistic Regression with TFIDF: ", f1_logRegTfidfClf*100)
print ("Accuracy of Logistic Regression with TFIDF: ", accuracy_logRegTfidfClf*100)


1. Count Vectorizer
F1-score of Logistic Regression:  97.10929609617548
Accuracy of Logistic Regression:  97.14908603052154

2. TFIDF Vectorizer
F1-score of Logistic Regression with TFIDF:  97.04153421164125
Accuracy of Logistic Regression with TFIDF:  97.06523561965453


### Build a Model - Support Vector Machine
1. Count Vectorizer
2. TFIDF Vectorizer
3. Count Vectorizer with Tuning
4. TFIDF Vectorizer with Tuning

In [6]:
print ("1. Count Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = False, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

svmClf = SVC()
svmClf.fit(X_train, y_train)
svmClf_ypred = svmClf.predict(X_test)
f1_svmClf = f1_score(y_test, svmClf_ypred, average = 'weighted')
accuracy_svmClf = accuracy_score(y_test, svmClf_ypred)
print ("F1-score of SVM: ", f1_svmClf*100)
print ("Accuracy of SVM: ", accuracy_svmClf*100)


print ("\n2. TFIDF Vectorizer")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = True, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

svmTfidfClf = LogisticRegression()
svmTfidfClf.fit(X_train, y_train)
svmTfidfClf_ypred = svmTfidfClf.predict(X_test)
f1_svmTfidfClf = f1_score(y_test, svmTfidfClf_ypred, average='weighted')
accuracy_svmTfidfClf = accuracy_score(y_test, svmTfidfClf_ypred)
print ("F1-score of SVM with TFIDF: ", f1_svmTfidfClf*100)
print ("Accuracy of SVM with TFIDF: ", accuracy_svmTfidfClf*100)

print ("\n3. Count Vectorizer with Tuning")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = False, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

# parameters = {'C':[1,2,3,4,5,6,7,8,14], 'gamma':[0.1, 0.01, 0.001, 0.0001], 'kernel':['linear', 'poly', 'rbf'], 'degree': [1,2,3,4,5]}
parameters = {'C':[1,2,3], 'gamma':[0.1, 0.01], 'kernel':['linear', 'poly', 'rbf'], 'degree': [1,2]}

svmClfTuned = GridSearchCV(estimator=SVC(), param_grid=parameters)
svmClfTuned.fit(X_train, y_train)
svmClfTuned_ypred = svmClfTuned.predict(X_test)
f1_svmClfTuned = f1_score(y_test, svmClfTuned_ypred, average = 'weighted')
accuracy_svmClfTuned = accuracy_score(y_test, svmClfTuned_ypred)
print ("F1-score of SVM with Tuning: ", f1_svmClfTuned*100)
print ("Accuracy of SVM with Tuning: ", accuracy_svmClfTuned*100)
print(svmClfTuned.best_params_)


print ("\n4. TFIDF Vectorizer with Tuning")

reviews = all_data_cleaned
topic_num = data['topic_num']
X_train, X_test, y_train, y_test = train_test_split(reviews, topic_num, test_size=0.25, random_state=42)

tfidfVectorizer = TfidfVectorizer(use_idf = True, min_df = 4, max_df=0.85)
X_train = tfidfVectorizer.fit_transform(X_train)
X_test = tfidfVectorizer.transform(X_test)

# parameters = {'C':[1,2,3,4,5,6,7,8,14], 'gamma':[0.1, 0.01, 0.001, 0.0001], 'kernel':['linear', 'poly', 'rbf'], 'degree': [1,2,3,4,5]}
parameters = {'C':[1,2,3], 'gamma':[0.1, 0.01], 'kernel':['linear', 'poly', 'rbf'], 'degree': [1,2]}

svmTfidfClfTuned = GridSearchCV(estimator=SVC(), param_grid=parameters)
svmTfidfClfTuned.fit(X_train, y_train)
svmTfidfClfTuned_ypred = svmTfidfClfTuned.predict(X_test)
f1_svmTfidfClfTuned = f1_score(y_test, svmTfidfClfTuned_ypred, average='weighted')
accuracy_svmTfidfClfTuned = accuracy_score(y_test, svmTfidfClfTuned_ypred)
print ("F1-score of SVM with TFIDF with Tuning: ", f1_svmTfidfClfTuned*100)
print ("Accuracy of SVM with TFIDF with Tuning: ", accuracy_svmTfidfClfTuned*100)
print (svmTfidfClfTuned.best_params_)


1. Count Vectorizer
F1-score of SVM:  71.75174864018979
Accuracy of SVM:  74.32500419252054

2. TFIDF Vectorizer
F1-score of SVM with TFIDF:  97.04153421164125
Accuracy of SVM with TFIDF:  97.06523561965453

3. Count Vectorizer with Tuning
F1-score of SVM with Tuning:  98.79102631018193
Accuracy of SVM with Tuning:  98.792554083515
{'C': 3, 'degree': 1, 'gamma': 0.1, 'kernel': 'linear'}

4. TFIDF Vectorizer with Tuning
F1-score of SVM with TFIDF with Tuning:  98.87544027095521
Accuracy of SVM with TFIDF with Tuning:  98.87640449438202
{'C': 3, 'degree': 1, 'gamma': 0.1, 'kernel': 'linear'}


### Summary of Accuracy

In [7]:
print ("F1-score of Multinomial Naive Bayes: ", f1_mnbClf*100)
print ("Accuracy of Multinomial Naive Bayes: ", accuracy_mnbClf*100)
print ("F1-score of Multinomial Naive Bayes with TFIDF: ", f1_mnbTfidfClf*100)
print ("Accuracy of Multinomial Naive Bayes with TFIDF: ", accuracy_mnbTfidfClf*100)
print ("\n")
print ("F1-score of Bernoulli Naive Bayes: ", f1_bnbClf*100)
print ("Accuracy of Bernoulli Naive Bayes: ", accuracy_bnbClf*100)
print ("F1-score of Bernoulli Naive Bayes with TFIDF: ", f1_bnbTfidfClf*100)
print ("Accuracy of Bernoulli Naive Bayes with TFIDF: ", accuracy_bnbTfidfClf*100)
print ("\n")
print ("F1-score of Logistic Regression: ", f1_logRegClf*100)
print ("Accuracy of Logistic Regression: ", accuracy_logRegClf*100)
print ("F1-score of Logistic Regression with TFIDF: ", f1_logRegTfidfClf*100)
print ("Accuracy of Logistic Regression with TFIDF: ", accuracy_logRegTfidfClf*100)
print ("\n")
print ("F1-score of SVM: ", f1_svmClf*100)
print ("Accuracy of SVM: ", accuracy_svmClf*100)
print ("F1-score of SVM with TFIDF: ", f1_svmTfidfClf*100)
print ("Accuracy of SVM with TFIDF: ", accuracy_svmTfidfClf*100)
print ("\n")
print ("F1-score of SVM with Tuning: ", f1_svmClfTuned*100)
print ("Accuracy of SVM with Tuning: ", accuracy_svmClfTuned*100)
print(svmClfTuned.best_params_)
print ("\n")
print ("F1-score of SVM with TFIDF with Tuning: ", f1_svmTfidfClfTuned*100)
print ("Accuracy of SVM with TFIDF with Tuning: ", accuracy_svmTfidfClfTuned*100)
print (svmTfidfClfTuned.best_params_)


F1-score of Multinomial Naive Bayes:  92.55332206254671
Accuracy of Multinomial Naive Bayes:  92.67147409022304
F1-score of Multinomial Naive Bayes with TFIDF:  92.8543552552533
Accuracy of Multinomial Naive Bayes with TFIDF:  92.93979540499748


F1-score of Bernoulli Naive Bayes:  90.91530178040284
Accuracy of Bernoulli Naive Bayes:  91.0950863659232
F1-score of Bernoulli Naive Bayes with TFIDF:  90.91530178040284
Accuracy of Bernoulli Naive Bayes with TFIDF:  91.0950863659232


F1-score of Logistic Regression:  97.10929609617548
Accuracy of Logistic Regression:  97.14908603052154
F1-score of Logistic Regression with TFIDF:  97.04153421164125
Accuracy of Logistic Regression with TFIDF:  97.06523561965453


F1-score of SVM:  71.75174864018979
Accuracy of SVM:  74.32500419252054
F1-score of SVM with TFIDF:  97.04153421164125
Accuracy of SVM with TFIDF:  97.06523561965453


F1-score of SVM with Tuning:  98.79102631018193
Accuracy of SVM with Tuning:  98.792554083515
{'C': 3, 'degree': 1,

### Manual check againt the classifier classification 
The classification done by the best model (tuned SVM with TFIDF) was saved to a csv file to do a manual check against the result

In [20]:
testing_predictions = []
testing_actual = []
for i in range(len((X_test))):
    if svmTfidfClfTuned_ypred[i] == 0:
        testing_predictions.append('delivery')
    elif svmTfidfClfTuned_ypred[i] == 1:
        testing_predictions.append('service')
    elif svmTfidfClfTuned_ypred[i] == 2:
        testing_predictions.append('price')
    elif svmTfidfClfTuned_ypred[i] == 3:
        testing_predictions.append('quality')
    else:
        testing_predictions.append('no_topic')
        
for i in range(len((X_test))):
    if  list(X_test)[i]== 0:
        testing_actual.append('delivery')
    elif list(X_test)[i] == 1:
        testing_actual.append('service')
    elif list(X_test)[i] == 2:
        testing_actual.append('price')
    elif list(X_test)[i] == 3:
        testing_actual.append('quality')
    else:
        testing_actual.append('no_topic')
        
check_df = pd.DataFrame({'actual label': list(y_test), 'prediction': testing_prediction, 'abstract':list(X_test)})
# check_df.to_csv('Data/check_df.csv')
check_df

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

### Import pickle and save the classifier and vectorizer to the pickle
The pickle can be use when new reviews are inserted into the model

In [ ]:
import pickle
pickle.dump(tfidfVectorizer, open("../topic_vectorizer.pickle", "wb"))
pickle.dump(svmTfidfClfTuned, open("../topic_classifier.pickle", 'wb'))